In [1]:
from mindspore import context
from mindspore.nn import Adam
from sponge import Sponge
from sponge import Molecule
from sponge import ForceField
from sponge import UpdaterMD
from sponge import WithEnergyCell
from sponge.control import VelocityVerlet, Langevin
from sponge.potential import SphericalRestrict
from sponge.function import VelocityGenerator
from sponge.callback import WriteH5MD, RunInfo

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

In [3]:
system = Molecule(template='water.spce.yaml')

In [4]:
system.reduplicate([0.3, 0, 0])
system.reduplicate([0, 0.3, 0])
new_sys = system.copy([0, 0, -0.3])
system.reduplicate([0, 0, 0.3])
system.append(new_sys)

Molecule<>

In [5]:
potential = ForceField(system, parameters='SPCE')

In [6]:
opt = Adam(system.trainable_params(), 1e-3)

In [7]:
mini = Sponge(system, potential, opt)

[MindSPONGE] Using fully connected neighbour list (not updated).


In [8]:
run_info = RunInfo(10)
mini.run(1000, callbacks=[run_info])

[MindSPONGE] Started simulation at 2023-05-18 15:46:01
[MindSPONGE] Step: 0, E_pot: 242.99136
[MindSPONGE] Step: 10, E_pot: 161.25085
[MindSPONGE] Step: 20, E_pot: 86.98923
[MindSPONGE] Step: 30, E_pot: 12.297862
[MindSPONGE] Step: 40, E_pot: -51.894093
[MindSPONGE] Step: 50, E_pot: -100.20014
[MindSPONGE] Step: 60, E_pot: -150.86668
[MindSPONGE] Step: 70, E_pot: -202.51653
[MindSPONGE] Step: 80, E_pot: -247.87825
[MindSPONGE] Step: 90, E_pot: -282.96652
[MindSPONGE] Step: 100, E_pot: -313.05664
[MindSPONGE] Step: 110, E_pot: -344.11502
[MindSPONGE] Step: 120, E_pot: -376.55453
[MindSPONGE] Step: 130, E_pot: -408.30453
[MindSPONGE] Step: 140, E_pot: -437.04016
[MindSPONGE] Step: 150, E_pot: -461.128
[MindSPONGE] Step: 160, E_pot: -479.91858
[MindSPONGE] Step: 170, E_pot: -493.87756
[MindSPONGE] Step: 180, E_pot: -504.3511
[MindSPONGE] Step: 190, E_pot: -512.0626
[MindSPONGE] Step: 200, E_pot: -517.5331
[MindSPONGE] Step: 210, E_pot: -521.54254
[MindSPONGE] Step: 220, E_pot: -524.7357
[

In [9]:
temp = 300
vgen = VelocityGenerator(temp)
velocity = vgen(system.shape, system.atom_mass)

In [10]:
opt = UpdaterMD(system,
                integrator=VelocityVerlet(system),
                thermostat=Langevin(system, temp),
                velocity=velocity,
                time_step=5e-4)

## 外加限制

在MindSPONGE的WithEnergyCell中可以自定义一些外加的势能项，这是一个非常灵活的操作，支持同时定义多个Bias叠加使用，并且在MindSpore框架的支持下都是可微的。比如本案例中施加的Bias是中心球形势SphericalRestrict，用于把原子的运动限制在一个球形范围内。另外MindSponge也支持原位谐振子势OscillatorBias，可以将原子限制在原始位置附近，这个功能在蛋白质预测中能够起到较大的作用。基于MindSPONGE的基础类PotentialCell，用户还可以自定义更多种类的Bias。

In [11]:
sim = WithEnergyCell(system, potential, bias=SphericalRestrict(radius=1.5, center=[0, 0, 0]))
md = Sponge(sim, optimizer=opt)

[MindSPONGE] Using fully connected neighbour list (not updated).


In [12]:
cb_h5md = WriteH5MD(system, 'tutorial_b04.h5md', save_freq=10, write_velocity=True, write_force=True)

In [13]:
md.run(3000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-05-18 15:46:17
[MindSPONGE] Step: 0, E_pot: -549.8801, E_kin: 123.11225, E_tot: -426.76788, Temperature: 282.03812
[MindSPONGE] Step: 10, E_pot: -515.7199, E_kin: 97.436646, E_tot: -418.28326, Temperature: 223.2178
[MindSPONGE] Step: 20, E_pot: -507.69943, E_kin: 88.4718, E_tot: -419.22763, Temperature: 202.68024
[MindSPONGE] Step: 30, E_pot: -477.35736, E_kin: 61.605106, E_tot: -415.75226, Temperature: 141.13126
[MindSPONGE] Step: 40, E_pot: -486.60107, E_kin: 81.03616, E_tot: -405.5649, Temperature: 185.64592
[MindSPONGE] Step: 50, E_pot: -474.58142, E_kin: 62.99297, E_tot: -411.58844, Temperature: 144.31071
[MindSPONGE] Step: 60, E_pot: -482.3633, E_kin: 74.6035, E_tot: -407.75983, Temperature: 170.90932
[MindSPONGE] Step: 70, E_pot: -467.71927, E_kin: 73.19846, E_tot: -394.5208, Temperature: 167.69052
[MindSPONGE] Step: 80, E_pot: -468.22095, E_kin: 74.11627, E_tot: -394.10468, Temperature: 169.79312
[MindSPONGE] Step: 90, E_pot: -459.90732, 